In [1]:
import numpy as np
import pandas as pd
import shapefile
from fuzzywuzzy import process
import json

In [2]:
# load netindex dataset
nidf = pd.read_csv('net_index/city_daily_speeds.csv')
nidf.head().T

c:\users\tomzh\miniconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
country,El Salvador,El Salvador,El Salvador,El Salvador,El Salvador
country_code,SV,SV,SV,SV,SV
region,San Salvador,San Salvador,San Salvador,San Salvador,San Salvador
region_code,10,10,10,10,10
city,San Salvador,San Salvador,San Salvador,San Salvador,San Salvador
date,2008-01-01,2008-01-02,2008-01-03,2008-01-04,2008-01-05
download_kbps,1121.51,1119.04,1120.72,1126.98,1123.01
upload_kbps,381.828,383.921,387.055,386.264,381.804
total_tests,4583,4631,4494,4331,4184
distance_miles,21.6485,21.3162,21.9008,22.0836,22.1806


In [3]:
# drop unwanted columns
nidf.drop(columns=['country_code', 'region_code'], inplace=True)
nidf.head().T

,0,1,2,3,4
country,El Salvador,El Salvador,El Salvador,El Salvador,El Salvador
region,San Salvador,San Salvador,San Salvador,San Salvador,San Salvador
city,San Salvador,San Salvador,San Salvador,San Salvador,San Salvador
date,2008-01-01,2008-01-02,2008-01-03,2008-01-04,2008-01-05
download_kbps,1121.51,1119.04,1120.72,1126.98,1123.01
upload_kbps,381.828,383.921,387.055,386.264,381.804
total_tests,4583,4631,4494,4331,4184
distance_miles,21.6485,21.3162,21.9008,22.0836,22.1806


In [4]:
# group by country and date and calculate mean
nidf = nidf.groupby(['country', 'date']).agg({
    'download_kbps':'mean',
    'upload_kbps':'mean',
    'total_tests':'sum',
    'distance_miles': 'mean'
}).reset_index()
nidf.head().T

,0,1,2,3,4
country,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan
date,2010-07-15,2010-07-16,2010-07-17,2010-07-18,2010-07-19
download_kbps,226.6,270.211,313.048,360.694,447.529
upload_kbps,105.9,106.895,114.143,109.863,105.736
total_tests,10,19,21,31,35
distance_miles,244.01,238.295,237.076,234.949,236.993


In [5]:
# validate country names
gsf = shapefile.Reader('./gadm36_levels_shp/gadm36_0')
gdf = pd.DataFrame([record[1] for record in gsf.iterRecords()], columns=['country'])

In [6]:
ok_count = 0
fail_count = 0

for country in nidf['country'].unique():
    if gdf[gdf['country'] == country].shape[0] != 0:
        ok_count += 1
    else:
        print(country)
        fail_count += 1

print('OK', ok_count, 'FAIL', fail_count)

Aland Islands
Brunei Darussalam
Cote D'Ivoire
Curacao
Iran, Islamic Republic of
Kazakstan
Korea, Republic of
Lao People's Democratic Republic
Libyan Arab Jamahiriya
Macau
Moldova, Republic of
Netherlands Antilles
Palestinian Territory
Russian Federation
Saint Martin
Syrian Arab Republic
Tanzania, United Republic of
Virgin Islands, British
OK 171 FAIL 18


In [7]:
# manually correct netindex dataset
nidf.replace({'country': 'Russian Federation'}, 'Russia', inplace=True)
nidf.replace({'country': 'Korea, Republic of'}, 'South Korea', inplace=True)
nidf.replace({'country': 'Kazakstan'}, 'Kazakhstan', inplace=True)
nidf.replace({'country': 'Iran, Islamic Republic of'}, 'Iran', inplace=True)
nidf.replace({'country': 'Palestinian Territory'}, 'Palestina', inplace=True)
nidf.replace({'country': 'Moldova, Republic of'}, 'Moldova', inplace=True)
nidf.replace({'country': 'Syrian Arab Republic'}, 'Syria', inplace=True)
nidf.replace({'country': 'Brunei Darussalam'}, 'Brunei', inplace=True)
nidf.replace({'country': 'Macau'}, 'Macao', inplace=True)
nidf.replace({'country': 'Netherlands Antilles'}, 'Netherlands', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Aruba'].index, inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Lao People\'s Democratic Republic'].index, inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Gibraltar'].index, inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Maldives'].index, inplace=True)
nidf.replace({'country': 'Virgin Islands, British'}, 'British Virgin Islands', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Anguilla'].index, inplace=True)
nidf.replace({'country': 'Tanzania, United Republic of'}, 'Tanzania', inplace=True)
nidf.replace({'country': 'Cote D\'Ivoire'}, 'Côte d\'Ivoire', inplace=True)
nidf.replace({'country': 'Aland Islands'}, 'Åland', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Curacao'].index, inplace=True)
nidf.replace({'country': 'Libyan Arab Jamahiriya'}, 'Libya', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Saint Martin'].index, inplace=True)

In [8]:
ok_count = 0
fail_count = 0

for country in nidf['country'].unique():
    if gdf[gdf['country'] == country].shape[0] != 0:
        ok_count += 1
    else:
        print(country)
        fail_count += 1

print('OK', ok_count, 'FAIL', fail_count)

OK 181 FAIL 0


In [9]:
country_names = [country[0] for country in nidf[['country']].value_counts().index.to_list()]
country_names[:5]

['Netherlands', 'Italy', 'Mexico', 'Malaysia', 'Macedonia']

In [10]:
nidf['date'].min()

'2008-01-01'

In [11]:
nidf['date'].max()

'2014-10-02'

In [12]:
idx = pd.date_range(nidf['date'].min(), nidf['date'].max())
idx

DatetimeIndex(['2008-01-01', '2008-01-02', '2008-01-03', '2008-01-04',
               '2008-01-05', '2008-01-06', '2008-01-07', '2008-01-08',
               '2008-01-09', '2008-01-10',
               ...
               '2014-09-23', '2014-09-24', '2014-09-25', '2014-09-26',
               '2014-09-27', '2014-09-28', '2014-09-29', '2014-09-30',
               '2014-10-01', '2014-10-02'],
              dtype='datetime64[ns]', length=2467, freq='D')

In [13]:
result = {}

In [14]:
cn = country_names[-1]
ts = pd.Series(dict(zip(nidf[nidf['country'] == cn]['date'], nidf[nidf['country'] == cn]['download_kbps'])))
ts.index = pd.DatetimeIndex(ts.index)
ts = ts.reindex(idx, fill_value=-1)
str(ts.index.to_list()[0])[:10]

'2008-01-01'

In [15]:
result['date'] = [str(date)[:10] for date in ts.index.to_list()]
print(result['date'][0])
print(result['date'][-1])

2008-01-01
2014-10-02


In [16]:
result['countries'] = {}

In [17]:
for cn in country_names:
    tmp = {}
    ts = pd.Series(dict(zip(nidf[nidf['country'] == cn]['date'], nidf[nidf['country'] == cn]['download_kbps'])))
    ts.index = pd.DatetimeIndex(ts.index)
    ts = ts.reindex(idx, fill_value=0)
    tmp['download_kbps'] = ts.to_list()
    ts = pd.Series(dict(zip(nidf[nidf['country'] == cn]['date'], nidf[nidf['country'] == cn]['upload_kbps'])))
    ts.index = pd.DatetimeIndex(ts.index)
    ts = ts.reindex(idx, fill_value=0)
    tmp['upload_kbps'] = ts.to_list()
    ts = pd.Series(dict(zip(nidf[nidf['country'] == cn]['date'], nidf[nidf['country'] == cn]['total_tests'])))
    ts.index = pd.DatetimeIndex(ts.index)
    ts = ts.reindex(idx, fill_value=0)
    tmp['total_tests'] = ts.to_list()
    result['countries'][cn] = tmp

In [18]:
with open('global_daily.json', 'w') as f:
    f.write(json.dumps(result))

In [19]:
# count days
dim_u = pd.Series([x[:7] for x in result['date']]).unique()
dim_vc = pd.Series([x[:7] for x in result['date']]).value_counts().to_dict()
dim = [dim_vc[x] for x in dim_u]
print(sum(dim))
print(dim[1], dim[13], dim[25], dim[37], dim[49], dim[61], dim[73])

2467
29 28 28 28 29 28 28


In [20]:
result2 = {
    'date': list(dim_u),
    'countries': {}
}

In [21]:
for cn in country_names:
    # per country
    pos = 0
    tmp = {
        'download_kbps': [],
        'upload_kbps': [],
        'total_tests': []
    }
    for d in dim:
        # per month
        down = []
        up = []
        total = 0
        for p in range(pos, pos + d):
            # per day
            if result['countries'][cn]['total_tests'][p] != 0:
                down.append(result['countries'][cn]['download_kbps'][p])
                up.append(result['countries'][cn]['upload_kbps'][p])
            total += result['countries'][cn]['total_tests'][p]
        pos += d
        if total != 0:
            tmp['download_kbps'].append(sum(down) / len(down))
            tmp['upload_kbps'].append(sum(up) / len(up))
        else:
            tmp['download_kbps'].append(0)
            tmp['upload_kbps'].append(0)
        tmp['total_tests'].append(total)
    result2['countries'][cn] = tmp

In [22]:
with open('global_monthly.json', 'w') as f:
    f.write(json.dumps(result2))